In [14]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Baixar o tokenizer do NLTK
nltk.download('punkt')

# Exemplo de dataset com descrições de produtos
data = {
    'produto': [
        "Smartphone iPhone 12 da Apple com 64GB de armazenamento.",
        "Notebook Dell Inspiron com 16GB de RAM e 1TB de SSD.",
        "Notebook Dell Latitude com 32GB de RAM e 1TB de SSD.",
        "Smartphone Samsung Galaxy S21 com câmera de 64MP.",
        "Televisão LG 55 polegadas 4K Ultra HD.",
        "Notebook Apple MacBook Air com M1 Chip.",
        "Câmera Canon EOS com lente de 24-70mm.",
        "Fone de ouvido Bluetooth Sony WH-1000XM4.",
        "Smartphone Xiaomi Mi 11 com Snapdragon 888.",
        "Tablet Samsung Galaxy Tab S6 com 128GB de armazenamento.",
        "Televisão Sony 65 polegadas 4K OLED."
    ]
}

df = pd.DataFrame(data)

# Tokenizar as descrições dos produtos
df['tokens'] = df['produto'].apply(lambda x: word_tokenize(x.lower()))

# Treinar o modelo Word2Vec nas descrições dos produtos
w2v_model = Word2Vec(sentences=df['tokens'], vector_size=100, window=5, min_count=1, workers=4)

# Função para converter uma descrição de produto em um vetor médio de Word2Vec
def get_average_vector(tokens_list, model):
    vectors = [model.wv[word] for word in tokens_list if word in model.wv]
    if len(vectors) > 0:
        return sum(vectors) / len(vectors)
    else:
        return [0] * model.vector_size

# Criar vetores médios para os produtos
df['vector'] = df['tokens'].apply(lambda x: get_average_vector(x, w2v_model))

# Função para recomendar produtos com base em similaridade de cosseno
def recomendar_produtos(produto_descricao, df, model, top_n=3):
    # Pré-processar e converter o produto em um vetor
    tokens = word_tokenize(produto_descricao.lower())
    produto_vector = get_average_vector(tokens, model)
    
    # Calcular a similaridade de cosseno entre o produto e todos os outros produtos
    df['similarity'] = df['vector'].apply(lambda x: cosine_similarity([produto_vector], [x])[0][0])
    
    # Ordenar os produtos pela similaridade
    produtos_similares = df.sort_values(by='similarity', ascending=False).head(top_n)
    
    # Retornar os produtos mais similares
    return produtos_similares[['produto', 'similarity']]

# Exemplo de uso: recomendar produtos semelhantes a um novo produto
novo_produto = "Smartphone da Apple com 128GB de armazenamento."
produtos_recomendados = recomendar_produtos(novo_produto, df, w2v_model, top_n=3)

# Exibir as recomendações
print(f"Produtos recomendados para '{novo_produto}':\n")
print(produtos_recomendados)


Produtos recomendados para 'Smartphone da Apple com 128GB de armazenamento.':

                                             produto  similarity
0  Smartphone iPhone 12 da Apple com 64GB de arma...    0.835941
9  Tablet Samsung Galaxy Tab S6 com 128GB de arma...    0.583094
5            Notebook Apple MacBook Air com M1 Chip.    0.530048


[nltk_data] Downloading package punkt to /home/alexmarino/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
df

,produto,tokens,vector,similarity
0,Smartphone iPhone 12 da Apple com 64GB de arma...,"[smartphone, iphone, 12, da, apple, com, 64gb,...","[-0.001135357, 0.0003248836, 0.00021996131, 0....",0.825904
1,Notebook Dell Inspiron com 16GB de RAM e 1TB d...,"[notebook, dell, inspiron, com, 16gb, de, ram,...","[0.0017830143, 0.00085552013, 0.0015023197, 0....",0.425541
2,Smartphone Samsung Galaxy S21 com câmera de 64MP.,"[smartphone, samsung, galaxy, s21, com, câmera...","[-0.0021626959, 0.0014249876, -0.00052829226, ...",0.508201
3,Televisão LG 55 polegadas 4K Ultra HD.,"[televisão, lg, 55, polegadas, 4k, ultra, hd, .]","[-0.00064801465, -0.0002821263, -0.0011818206,...",0.220843
4,Notebook Apple MacBook Air com M1 Chip.,"[notebook, apple, macbook, air, com, m1, chip, .]","[-0.0012537302, -9.07224e-05, -1.465861e-05, 0...",0.439733
5,Câmera Canon EOS com lente de 24-70mm.,"[câmera, canon, eos, com, lente, de, 24-70mm, .]","[-0.0028462987, 0.0012128465, -0.00092612853, ...",0.349317
6,Fone de ouvido Bluetooth Sony WH-1000XM4.,"[fone, de, ouvido, bluetooth, sony, wh-1000xm4...","[0.0025557524, 0.0014946843, 0.0011683785, -0....",0.326656
7,Smartphone Xiaomi Mi 11 com Snapdragon 888.,"[smartphone, xiaomi, mi, 11, com, snapdragon, ...","[-0.0024049887, 0.0018255042, 0.0016637945, -2...",0.251469
8,Tablet Samsung Galaxy Tab S6 com 128GB de arma...,"[tablet, samsung, galaxy, tab, s6, com, 128gb,...","[-0.00089090376, -1.48666995e-05, -0.001931852...",0.504061
9,Televisão Sony 65 polegadas 4K OLED.,"[televisão, sony, 65, polegadas, 4k, oled, .]","[-0.0017131099, 0.0019378832, 0.0018897967, 0....",0.169112


In [19]:
# Exemplo de uso: recomendar produtos semelhantes a um novo produto
novo_produto = "Notebook Dell atitude"
produtos_recomendados = recomendar_produtos(novo_produto, df, w2v_model, top_n=5)

# Exibir as recomendações
print(f"Produtos recomendados para '{novo_produto}':\n")
print(produtos_recomendados)


Produtos recomendados para 'Notebook Dell atitude':

                                             produto  similarity
5            Notebook Apple MacBook Air com M1 Chip.    0.429334
2  Notebook Dell Latitude com 32GB de RAM e 1TB d...    0.354187
1  Notebook Dell Inspiron com 16GB de RAM e 1TB d...    0.337985
0  Smartphone iPhone 12 da Apple com 64GB de arma...    0.119259
7          Fone de ouvido Bluetooth Sony WH-1000XM4.    0.061691
